In [1]:
from neo4j import GraphDatabase
import spacy
import gradio as gr


/home/raghu/anaconda3/envs/neo4j/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Start Neo4j services and create a blank project.

If it is Neo4j sandbox, login and then enter related information below. Also ensure to create a blank project.

In [2]:
driver = GraphDatabase.driver("bolt://23.22.35.116:7687", 
                              auth=("neo4j", 
                                    "seamanship-stare-branches"))
    

In [4]:
nlp = spacy.load("en_core_web_sm")


#### Functions

In [5]:
def extract_entity(question, entity_type):
    doc = nlp(question)
    for ent in doc.ents:
        if ent.label_ in entity_type:
            return ent.text
    return None


In [6]:
def get_answer(question):
    doc = nlp(question)
    movie_title = extract_entity(question, ["WORK_OF_ART", "MOVIE"])
    director_name = extract_entity(question, ["PERSON"])
    if movie_title is None:
        return "Sorry, I don't understand what movie you're asking about."
    
    with driver.session() as session:
        if "release" in question or "released" in question:
            result = session.run(f"MATCH (m:Movie) WHERE m.title = '{movie_title}' RETURN m.released")
            if result.peek() is None:
                return f"No information found for the movie '{movie_title}'"
            for record in result:
                return f"The release date of '{movie_title}' is: {record['m.released']}"
        else:
            return "Sorry, I don't understand what you're asking."


In [7]:
def chatbot(input, history=[]):
    output = get_answer(input)
    history.append((input, output))
    return history, history


In [ ]:
gr.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state']).launch(debug = True)
